In [3]:
!pip install Flask pandas scikit-learn

from flask import Flask, render_template_string, request, jsonify
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from math import expm1
# Import train_test_split
from sklearn.model_selection import train_test_split

# Initialize Flask app
app = Flask(__name__)

# Load the trained model and scaler (you can save and load these in real applications)
scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')
ridge = Ridge(alpha=1.0)

# Preload the model with your data (this is simplified, normally save and load the model)
train = pd.read_csv('/content/train.csv')  # Assuming 'train (1).csv' is in the current directory or provide the correct path
columns_to_keep = ['LotArea', 'BedroomAbvGr', 'BsmtFullBath', 'BsmtHalfBath', 'HalfBath', 'TotalBsmtSF', 'FullBath', 'SalePrice']
train = train[columns_to_keep]
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']
X_scaled = scaler.fit_transform(X)
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, random_state=42, test_size=0.33)

# Fit the Ridge regression model
ridge.fit(X_train_scaled, y_train)

# Home route to display the form
@app.route('/')
def home():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Sale Price Prediction</title>
    </head>
    <body>
        <div style="text-align: center;">
            <h1>Predict Sale Price for a House</h1>

            <!-- Input Form -->
            <form action="/predict" method="POST">
                <label for="LotArea">Lot Area:</label>
                <input type="number" step="any" name="LotArea" required><br><br>

                <label for="BedroomAbvGr">Bedrooms Above Ground:</label>
                <input type="number" step="any" name="BedroomAbvGr" required><br><br>

                <label for="BsmtFullBath">Basement Full Bathrooms:</label>
                <input type="number" step="any" name="BsmtFullBath" required><br><br>

                <label for="BsmtHalfBath">Basement Half Bathrooms:</label>
                <input type="number" step="any" name="BsmtHalfBath" required><br><br>

                <label for="HalfBath">Half Bathrooms:</label>
                <input type="number" step="any" name="HalfBath" required><br><br>

                <label for="TotalBsmtSF">Total Basement Square Feet:</label>
                <input type="number" step="any" name="TotalBsmtSF" required><br><br>

                <label for="FullBath">Full Bathrooms:</label>
                <input type="number" step="any" name="FullBath" required><br><br>

                <button type="submit">Predict Sale Price</button>
            </form>

            <h2>{{ prediction_text }}</h2>
        </div>
    </body>
    </html>
    ''')

# Predict route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get user inputs from the form
        lot_area = float(request.form['LotArea'])
        bedroom_abv_gr = float(request.form['BedroomAbvGr'])
        bsmt_full_bath = float(request.form['BsmtFullBath'])
        bsmt_half_bath = float(request.form['BsmtHalfBath'])
        half_bath = float(request.form['HalfBath'])
        total_bsmt_sf = float(request.form['TotalBsmtSF'])
        full_bath = float(request.form['FullBath'])

        # Create input features as a numpy array
        features = np.array([[lot_area, bedroom_abv_gr, bsmt_full_bath, bsmt_half_bath, half_bath, total_bsmt_sf, full_bath]])

        # Apply the same transformations to the input features
        features_scaled = scaler.transform(features)
        features_imputed = imputer.transform(features_scaled)

        # Make prediction
        prediction = ridge.predict(features_imputed)
        predicted_sale_price = expm1(prediction[0])

        return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Sale Price Prediction</title>
        </head>
        <body>
            <div style="text-align: center;">
                <h1>Predict Sale Price for a House</h1>

                <!-- Input Form -->
                <form action="/predict" method="POST">
                    <label for="LotArea">Lot Area:</label>
                    <input type="number" step="any" name="LotArea" required><br><br>

                    <label for="BedroomAbvGr">Bedrooms Above Ground:</label>
                    <input type="number" step="any" name="BedroomAbvGr" required><br><br>

                    <label for="BsmtFullBath">Basement Full Bathrooms:</label>
                    <input type="number" step="any" name="BsmtFullBath" required><br><br>

                    <label for="BsmtHalfBath">Basement Half Bathrooms:</label>
                    <input type="number" step="any" name="BsmtHalfBath" required><br><br>

                    <label for="HalfBath">Half Bathrooms:</label>
                    <input type="number" step="any" name="HalfBath" required><br><br>

                    <label for="TotalBsmtSF">Total Basement Square Feet:</label>
                    <input type="number" step="any" name="TotalBsmtSF" required><br><br>

                    <label for="FullBath">Full Bathrooms:</label>
                    <input type="number" step="any" name="FullBath" required><br><br>

                    <button type="submit">Predict Sale Price</button>
                </form>

                <h2>Predicted Sale Price: ${{ predicted_sale_price:,.2f }}</h2>
            </div>
        </body>
        </html>
        '''.format(predicted_sale_price=predicted_sale_price))

    except Exception as e:
        return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Sale Price Prediction</title>
        </head>
        <body>
            <div style="text-align: center;">
                <h1>Predict Sale Price for a House</h1>

                <!-- Input Form -->
                <form action="/predict" method="POST">
                    <label for="LotArea">Lot Area:</label>
                    <input type="number" step="any" name="LotArea" required><br><br>

                    <label for="BedroomAbvGr">Bedrooms Above Ground:</label>
                    <input type="number" step="any" name="BedroomAbvGr" required><br><br>

                    <label for="BsmtFullBath">Basement Full Bathrooms:</label>
                    <input type="number" step="any" name="BsmtFullBath" required><br><br>

                    <label for="BsmtHalfBath">Basement Half Bathrooms:</label>
                    <input type="number" step="any" name="BsmtHalfBath" required><br><br>

                    <label for="HalfBath">Half Bathrooms:</label>
                    <input type="number" step="any" name="HalfBath" required><br><br>

                    <label for="TotalBsmtSF">Total Basement Square Feet:</label>
                    <input type="number" step="any" name="TotalBsmtSF" required><br><br>

                    <label for="FullBath">Full Bathrooms:</label>
                    <input type="number" step="any" name="FullBath" required><br><br>

                    <button type="submit">Predict Sale Price</button>
                </form>

                <h2>Error: {{ error_message }}</h2>
            </div>
        </body>
        </html>
        '''.format(error_message=str(e)))

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
